In [89]:
import pandas as pd
import pandas_explode 
pandas_explode.patch()
#import datetime
#import os

In [90]:
ct_csv = "Resources/CT_data.csv"
ct_df = pd.read_csv(ct_csv)
ct_df.head(3)

,Rank,Title,Status,Study Results,Conditions,Interventions,Sponsor/Collaborators,Phases,Funded Bys,Study Type,Results First Posted,URL
0,1,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Completed,Has Results,Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Phase 3,Industry,Interventional,26-Jul-17,https://ClinicalTrials.gov/show/NCT02756624
1,2,A Phase III Study to Demonstrate the Safety an...,Completed,Has Results,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Phase 3,Industry,Interventional,27-Feb-19,https://ClinicalTrials.gov/show/NCT02334436
2,3,A Phase III Study to Demonstrate the Safety an...,Completed,Has Results,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Phase 3,Industry,Interventional,26-Feb-19,https://ClinicalTrials.gov/show/NCT02334423


In [91]:
ct_df_clean = ct_df.drop(['Rank', 'Status', 'Study Results', 'Phases', 'Funded Bys'], axis=1)
ct_df_rename = ct_df_clean.rename(columns={"Sponsor/Collaborators":"Companies"})
ct_df_rename.head(3)

,Title,Conditions,Interventions,Companies,Study Type,Results First Posted,URL
0,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,https://ClinicalTrials.gov/show/NCT02756624
1,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Interventional,27-Feb-19,https://ClinicalTrials.gov/show/NCT02334436
2,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Interventional,26-Feb-19,https://ClinicalTrials.gov/show/NCT02334423


In [16]:
#ct_df_1nf = ct_df_clean.explode('Sponsor/Collaborators', )
#ct_df_1nf.head(3)

In [92]:
# new data frame to split URL columns 
new = ct_df_rename["URL"].str.split("w/", expand = True)
new.head(3)
ct_df_rename["CTID"]= new[1]
ct_df_rename.drop(columns =["URL"], inplace = True) 
ct_df_rename.head(3)

,Title,Conditions,Interventions,Companies,Study Type,Results First Posted,CTID
0,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624
1,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Interventional,27-Feb-19,NCT02334436
2,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Interventional,26-Feb-19,NCT02334423


In [98]:
# create a new dataframe to split the companies column into multiple rows
new2 = pd.DataFrame(ct_df_rename['Companies'].str.split('|').tolist(), index=ct_df_rename.CTID).stack()
new2.head(3)

CTID          
NCT02756624  0    Nicox Ophthalmics, Inc.
             1                  ORA, Inc.
NCT02334436  0               Evolus, Inc.
dtype: object

In [99]:
new2 = new2.reset_index([0, 'CTID'])
new2.head(3)

,CTID,0
0,NCT02756624,"Nicox Ophthalmics, Inc."
1,NCT02756624,"ORA, Inc."
2,NCT02334436,"Evolus, Inc."


In [100]:
new2.columns = ['CTID', 'Company']
new2.head(3)


,CTID,Company
0,NCT02756624,"Nicox Ophthalmics, Inc."
1,NCT02756624,"ORA, Inc."
2,NCT02334436,"Evolus, Inc."


In [96]:
ct_split1 = ct_df_rename.merge(new_df, left_on = 'CTID', right_on = 'CTID', how='outer')
ct_split1.head(3)

,Title,Conditions,Interventions,Companies,Study Type,Results First Posted,CTID,Company
0,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624,"Nicox Ophthalmics, Inc."
1,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624,"ORA, Inc."
2,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,"Biological: Botulinum toxin, Type A|Other: 0.9...","Evolus, Inc.|PPD",Interventional,27-Feb-19,NCT02334436,"Evolus, Inc."


In [97]:
# create a new dataframe to split the interventions column into multiple rows
new_split = pd.DataFrame(ct_df_rename['Interventions'].str.split('|').tolist(), index=ct_df_rename.CTID).stack()
new_split.head(3)

CTID          
NCT02756624  0                     Drug: AC-170 0.24%
             1                   Drug: AC-170 Vehicle
NCT02334436  0    Biological: Botulinum toxin, Type A
dtype: object

In [101]:
new_split = new_split.reset_index([0, 'CTID'])
new_split.head(3)

,CTID,0
0,NCT02756624,Drug: AC-170 0.24%
1,NCT02756624,Drug: AC-170 Vehicle
2,NCT02334436,"Biological: Botulinum toxin, Type A"


In [102]:
new_split.columns = ['CTID', 'Intervention']
new_split.head(3)

,CTID,Intervention
0,NCT02756624,Drug: AC-170 0.24%
1,NCT02756624,Drug: AC-170 Vehicle
2,NCT02334436,"Biological: Botulinum toxin, Type A"


In [103]:
ct_split2 = ct_split1.merge(new_split, left_on = 'CTID', right_on = 'CTID', how='outer')
ct_split2.head(3)

,Title,Conditions,Interventions,Companies,Study Type,Results First Posted,CTID,Company,Intervention
0,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624,"Nicox Ophthalmics, Inc.",Drug: AC-170 0.24%
1,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624,"Nicox Ophthalmics, Inc.",Drug: AC-170 Vehicle
2,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Drug: AC-170 0.24%|Drug: AC-170 Vehicle,"Nicox Ophthalmics, Inc.|ORA, Inc.",Interventional,26-Jul-17,NCT02756624,"ORA, Inc.",Drug: AC-170 0.24%


In [104]:
ct_final = ct_split2.drop(['Companies', 'Interventions'], axis=1)
ct_final.head()

,Title,Conditions,Study Type,Results First Posted,CTID,Company,Intervention
0,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,NCT02756624,"Nicox Ophthalmics, Inc.",Drug: AC-170 0.24%
1,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,NCT02756624,"Nicox Ophthalmics, Inc.",Drug: AC-170 Vehicle
2,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,NCT02756624,"ORA, Inc.",Drug: AC-170 0.24%
3,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,NCT02756624,"ORA, Inc.",Drug: AC-170 Vehicle
4,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,Interventional,27-Feb-19,NCT02334436,"Evolus, Inc.","Biological: Botulinum toxin, Type A"


In [108]:
ct_final2 = ct_final.set_index('CTID')
ct_final2.head()

,Title,Conditions,Study Type,Results First Posted,Company,Intervention
CTID,,,,,,
NCT02756624,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,"Nicox Ophthalmics, Inc.",Drug: AC-170 0.24%
NCT02756624,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,"Nicox Ophthalmics, Inc.",Drug: AC-170 Vehicle
NCT02756624,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,"ORA, Inc.",Drug: AC-170 0.24%
NCT02756624,"A Multi-Center, Vehicle-Controlled, Parallel-G...",Healthy Subjects,Interventional,26-Jul-17,"ORA, Inc.",Drug: AC-170 Vehicle
NCT02334436,A Phase III Study to Demonstrate the Safety an...,Glabellar Frown Lines,Interventional,27-Feb-19,"Evolus, Inc.","Biological: Botulinum toxin, Type A"


In [111]:
ct_final2.to_csv(r'Resources/ct_final2.csv')